# 6.3 クラスの分布が偏っている場合

分類ラベルの分布が偏っている場合の対処法

## アンダーサンプリング

例えば異常検知など正例が異常に少ない場合、負例の一部のみを使用してモデルを学習させる方法。  
また、異なる負例を取り出して学習させた複数のモデルを平均する手法 (バギング) も有効。  
[`imbalanced-learn`](https://imbalanced-learn.org/stable/) というライブラリを使える。(参考: [LightGBMでdownsampling+bagging](https://upura.hatenablog.com/entry/2019/01/12/193000))

kaggle の [TalkingData AdTracking Fraud Detection Challenge](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection) というコンペでは、1 億件以上のデータのうち正例は 0.2 % 程度であり、
1 位の解法はほとんどの負例を捨てて精度を出していた。

In [ ]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_x は学習データ、train_y は目的変数、test_x はテストデータ
train = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

# downsampling
# 正例の数と負例の数を揃える
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler(random_state=42)
x_resampled, y_resampled = sampler.fit_resample(train_x, train_y)
print(len(train[train['target']==1])*2, len(x_resampled))

## 特に工夫をしない

特に工夫をしなくても十分な精度が出ることも多い。

## 重み付け

parameter 更新の際の正例と負例の寄与の合計が等しくなるように正例に高い weight を指定する方法。

## オーバーサンプリング

負例の方が多い場合に、正例を増やして学習をさせる方法。  
Synthetic Minority Oversampling Technique (SMOTE) といった手法がある。

SMOTE とは、少数クラスのデータ点と近傍のデータ点間の単純な内挿によって新しいデータを生成する手法。  
拡張版が色々ある。

## 確率を予測する必要がある場合の注意点

評価指標が logloss などで適切な確率を予測する必要がある場合、正例と負例の比率を変えた場合は確率の補正を忘れずに行うように。  
(2.5.4 確率の予測値とその調整 を参照)

分析コンペでは under sampling or 何もしない手法が主に使われており、over sampling はあまり使われることはない。